In [20]:
%run utils.ipynb

In [21]:
tech = ['HCLTECH', 'WIPRO', 'INFY', 'TCS', 'TECHM']
banks = ['HDFCBANK', 'ICICIBANK', 'SBIN', 'KOTAKBANK', 'AXISBANK', 'FEDERALBNK', 'KARURVYSYA', 'CANBK']
others = ['RELIANCE', 'HINDUNILVR', 'ITC', 'M&M', 'TATAMOTORS', 'TATASTEEL', 'CIPLA']

_all = tech + banks + others

In [22]:
final_df = pd.concat([analyse(i)[0] for i in _all]).reset_index(drop=True)

In [23]:
final_df

,10_cross_30,MACD_Signal_MACD,MACD_lim,abv_50,abv_200,actual_returns,proba,stock
0,1,0,1,0,1,3.08,0.71,HCLTECH
1,1,1,0,0,0,2.69,0.74,HCLTECH
2,0,1,1,1,0,1.13,1.00,HCLTECH
3,0,1,0,1,0,5.70,1.00,WIPRO
4,0,1,1,1,0,4.07,0.92,WIPRO
...,...,...,...,...,...,...,...,...
57,1,1,0,1,0,2.59,1.00,TATAMOTORS
58,0,1,1,0,1,1.51,0.69,TATAMOTORS
59,0,1,0,1,0,4.13,1.00,TATASTEEL
60,0,1,1,1,0,1.74,0.94,TATASTEEL


In [24]:
summary = final_df.groupby(['10_cross_30', 'MACD_Signal_MACD', 'MACD_lim', 'abv_50', 'abv_200']).agg(
    avg_return = ('actual_returns', 'mean'),
    median_return = ('actual_returns', 'median'),
    avg_probability = ('proba', 'mean'),
    stocks = ('stock', 'unique'),
    stock_count = ('stock', 'count')
)

summary.sort_values(by=['stock_count', 'avg_probability', 'avg_return'], ascending=False)

avg_return  \
10_cross_30 MACD_Signal_MACD MACD_lim abv_50 abv_200               
0           1                0        1      0          4.596667   
1           0                0        0      0          2.220000   
0           1                0        1      1          4.470000   
                             1        1      0          2.416000   
1           0                0        0      1          1.388000   
                             1        0      1          2.280000   
                             0        1      0          2.290000   
            1                0        0      1          2.772500   
                                      1      0          3.776667   
            0                1        0      0          2.853333   
0           1                1        0      0          2.706667   
                                      1      1          1.256667   
1           1                0        0      0          2.456667   
0           1                1        0      1          1.590000   
1           0                0        1      1          2.115000   
            1                0        1      1          7.780000   
                             1        0      0          2.070000   

                                                      median_return  \
10_cross_30 MACD_Signal_MACD MACD_lim abv_50 abv_200                  
0           1                0        1      0                4.465   
1           0                0        0      0                2.115   
0           1                0        1      1                3.650   
                             1        1      0                1.740   
1           0                0        0      1                1.410   
                             1        0      1                2.440   
                             0        1      0                2.010   
            1                0        0      1                2.965   
                                      1      0                2.590   
            0                1        0      0                3.040   
0           1                1        0      0                2.820   
                                      1      1                1.070   
1           1                0        0      0                2.690   
0           1                1        0      1                1.510   
1           0                0        1      1                2.115   
            1                0        1      1                7.780   
                             1        0      0                2.070   

                                                      avg_probability  \
10_cross_30 MACD_Signal_MACD MACD_lim abv_50 abv_200                    
0           1                0        1      0               1.000000   
1           0                0        0      0               0.746667   
0           1                0        1      1               0.950000   
                             1        1      0               0.892000   
1           0                0        0      1               0.766000   
                             1        0      1               0.726000   
                             0        1      0               0.740000   
            1                0        0      1               0.735000   
                                      1      0               0.890000   
            0                1        0      0               0.870000   
0           1                1        0      0               0.773333   
                                      1      1               0.716667   
1           1                0        0      0               0.703333   
0           1                1        0      1               0.683333   
1           0                0        1      1               0.820000   
            1                0        1      1               1.000000   
                             1        0      0               0.660000   

                       